In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup

### Obtención de la lista de jugadores en la NBA desde el año 2000

In [2]:
# Busqueda de todos los nombres de jugadores desde el año 2000

data = []
count = 1
for year in range(2000,2020):
    website_url = requests.get('https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_per_game.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':"per_game_stats"})
    tabledata = My_table.findAll('td')
    for cell in tabledata:
        count = count+1
        if (count%29 == 2):
            data.append(str(year))
        data.append(cell.get_text())


In [3]:
import numpy as np
data2 = np.array(data)
refined = np.reshape(data2, (-1, 30))

### Creación de DataFrame con todos los jugadores y sus estadisticas en NBA

In [4]:
import pandas as pd
players = pd.DataFrame(refined)
players.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,2000,Tariq Abdul-Wahad,SG,25,TOT,61,56,25.9,4.5,10.6,...,.756,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4
1,2000,Tariq Abdul-Wahad,SG,25,ORL,46,46,26.2,4.8,11.2,...,.762,1.7,3.5,5.2,1.6,1.2,0.3,1.9,2.5,12.2
2,2000,Tariq Abdul-Wahad,SG,25,DEN,15,10,24.9,3.4,8.7,...,.738,1.6,1.9,3.5,1.7,0.4,0.8,1.3,2.1,8.9
3,2000,Shareef Abdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,...,.809,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3
4,2000,Cory Alexander,PG,26,DEN,29,2,11.3,1.0,3.4,...,.773,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8


In [5]:
# Creamos los nombres de los jugadores en formato lista, cambiamos el '*' que viene al final de los jugadores que han sido Hall of Fame y vemos la longitud de la misma

players = list(set(players[1]))
players = [player.replace("*", "") for player in players]
len(players)

1911

### Obtención de sus estadísticas universitarias

In [7]:
# Hacemos scraping de cada jugador de sus estadisticas universitarias

maindata = []
n=0
for name in players:
    realname = name
    name = name.lower()
    name = name.replace(" ", "-")
    name = name.replace(".", "")
    name = name.replace("\'", "")
    name = name.replace("’", "")
    name = name.replace("JR", "")
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':'players_per_game'})
    if My_table is not None:
        maindata = maindata + [realname]
        tabledata = My_table.findAll('td')
        data = []
        for cell in tabledata:
            data.append(cell.get_text())
        maindata = maindata +data[-28:]
    n+=1
    if n%100==0:
        print('Element counter: {}'.format(n))

Element counter: 100
Element counter: 200
Element counter: 300
Element counter: 400
Element counter: 500
Element counter: 600
Element counter: 700
Element counter: 800
Element counter: 900
Element counter: 1000
Element counter: 1100
Element counter: 1200
Element counter: 1300
Element counter: 1400
Element counter: 1500
Element counter: 1600
Element counter: 1700
Element counter: 1800
Element counter: 1900


In [8]:
seguro = maindata

In [9]:
# Separamos la información en cada columna

import numpy as np
maindata2 = np.array(maindata)
refined2 = np.reshape(maindata2, (-1, 29))
refined2

array([['Greg Monroe', 'Georgetown', '', ..., '14.5', '', '9.26'],
       ['Jarron Collins', 'Stanford', '', ..., '8.4', '', '8.87'],
       ['Jordan Adams', 'UCLA', '', ..., '16.4', '', '7.72'],
       ...,
       ['Pierre Jackson', 'Baylor', '', ..., '16.7', '', '7.58'],
       ['Jason Williams', 'Florida', '', ..., '17.1', '', ''],
       ['Jamaal Magloire', 'Kentucky', '', ..., '7.3', '', '10.42']],
      dtype='<U24')

In [10]:
# Le añadimos los colheaders traídos del anterior scraping del draft'19 y creados el DataFrame

colheaders = ['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '\xa0', 'SOS']

import pandas as pd
college_NBAplayers = pd.DataFrame(refined2, columns=colheaders)
college_NBAplayers.head()


,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,,SOS
0,Greg Monroe,Georgetown,,65,65,32.6,5.3,9.8,.543,5.2,...,6.1,8.2,3.2,1.5,1.5,2.9,2.5,14.5,,9.26
1,Jarron Collins,Stanford,,129,34,24.5,2.9,5.5,.520,2.8,...,4.7,5.5,1.1,0.6,0.4,1.3,1.8,8.4,,8.87
2,Jordan Adams,UCLA,,69,63,30.2,5.4,11.6,.467,4.0,...,3.3,4.6,2.1,2.4,0.2,1.4,2.1,16.4,,7.72
3,Reggie Jackson,Boston College,,99,52,28.0,4.6,9.9,.465,3.4,...,3.0,4.4,3.5,0.8,0.5,2.1,1.7,12.7,,7.25
4,James Webb,Boise State,,63,58,29.0,5.0,9.6,.518,3.8,...,6.4,8.6,0.8,1.3,0.6,1.3,2.8,13.5,,2.25


### Limpieza de valores nulos y columnas sin información

In [11]:
college_NBAplayers = college_NBAplayers.drop(['\xa0'], axis=1)

In [12]:
# Eliminamos las columnas sin información del DataFrame

col = college_NBAplayers.columns
cols = []
for co in col:
    cols.append(co)
cols.remove('Name')
cols.remove('School')
cols.remove('Conf')
print(cols)

['G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS']


In [13]:
# Limpiamos las columnas donde haya algún valor nulo ya que no nos sirven

# allstars_data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

for col in cols:
    college_NBAplayers[col] = pd.to_numeric(college_NBAplayers[col], errors='coerce')

college_NBAplayers = college_NBAplayers.dropna()
college_NBAplayers = college_NBAplayers.drop(['Conf'], axis=1)
college_NBAplayers.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS
0,Greg Monroe,Georgetown,65.0,65.0,32.6,5.3,9.8,0.543,5.2,9.3,...,2.1,6.1,8.2,3.2,1.5,1.5,2.9,2.5,14.5,9.26
1,Jarron Collins,Stanford,129.0,34.0,24.5,2.9,5.5,0.520,2.8,5.4,...,2.0,4.7,5.5,1.1,0.6,0.4,1.3,1.8,8.4,8.87
2,Jordan Adams,UCLA,69.0,63.0,30.2,5.4,11.6,0.467,4.0,7.3,...,1.3,3.3,4.6,2.1,2.4,0.2,1.4,2.1,16.4,7.72
3,Reggie Jackson,Boston College,99.0,52.0,28.0,4.6,9.9,0.465,3.4,6.4,...,1.4,3.0,4.4,3.5,0.8,0.5,2.1,1.7,12.7,7.25
4,James Webb,Boise State,63.0,58.0,29.0,5.0,9.6,0.518,3.8,6.1,...,2.2,6.4,8.6,0.8,1.3,0.6,1.3,2.8,13.5,2.25


In [14]:
# Vemos la longitud del DataFrame para comprobarla con la longitud de la lista de jugadores y ver cuantos hemos hecho bien el scraping

college_NBAplayers.shape

(897, 27)

### Añadimos la columna AllStar

In [15]:
# Archivo txt descargado de 

file = '../input/allstar.txt'
allstar_info = pd.read_csv(file)
allstar_info.head()

,Player,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,STL,BLK,TOV,PF,PTS,Unnamed: 25,MP.1,PTS.1,TRB.1,AST.1
0,Kareem Abdul-Jabbar,18,13.0,449.0,105,213,0.493,0.0,1.0,0.000,...,6.0,31.0,28.0,57,251,NaN,24.9,13.9,8.3,2.8
1,LeBron James,15,15.0,441.0,150,283,0.530,36.0,106.0,0.340,...,18.0,4.0,48.0,17,362,NaN,29.4,24.1,6.4,5.9
2,Tim Duncan,15,12.0,311.0,63,115,0.548,1.0,4.0,0.250,...,13.0,8.0,31.0,16,140,NaN,20.7,9.3,9.1,2.1
3,Kobe Bryant,15,15.0,415.0,119,238,0.500,22.0,68.0,0.324,...,38.0,6.0,35.0,35,290,NaN,27.7,19.3,5.0,4.7
4,Dirk Nowitzki,14,2.0,227.0,49,109,0.450,10.0,34.0,0.294,...,10.0,5.0,14.0,8,122,NaN,16.2,8.7,3.7,1.1


In [16]:
# Función para que ponga "Yes" si un jugador de la tabla "draft_info" ha sido seleccionado para jugar el partido de las estrellas

def allstar(college_NBAplayers, allstar_info):
    allstar_list = list(allstar_info['Player'])
    all_star=[]
    for i in college_NBAplayers['Name']:
        if i in allstar_list:
            all_star.append('Yes')
        else:
            all_star.append('No')
    college_NBAplayers['Allstar'] = all_star
    return college_NBAplayers

college_NBAplayers = allstar(college_NBAplayers, allstar_info)

In [17]:
college_NBAplayers.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS,Allstar
0,Greg Monroe,Georgetown,65.0,65.0,32.6,5.3,9.8,0.543,5.2,9.3,...,6.1,8.2,3.2,1.5,1.5,2.9,2.5,14.5,9.26,No
1,Jarron Collins,Stanford,129.0,34.0,24.5,2.9,5.5,0.520,2.8,5.4,...,4.7,5.5,1.1,0.6,0.4,1.3,1.8,8.4,8.87,No
2,Jordan Adams,UCLA,69.0,63.0,30.2,5.4,11.6,0.467,4.0,7.3,...,3.3,4.6,2.1,2.4,0.2,1.4,2.1,16.4,7.72,No
3,Reggie Jackson,Boston College,99.0,52.0,28.0,4.6,9.9,0.465,3.4,6.4,...,3.0,4.4,3.5,0.8,0.5,2.1,1.7,12.7,7.25,No
4,James Webb,Boise State,63.0,58.0,29.0,5.0,9.6,0.518,3.8,6.1,...,6.4,8.6,0.8,1.3,0.6,1.3,2.8,13.5,2.25,No


In [18]:
# Vemos cuantos valores tenemos de Allstar sobre el total del DataSet

college_NBAplayers['Allstar'].value_counts()

No     837
Yes     60
Name: Allstar, dtype: int64

### Añadimos columna con su posición en el Draft y año de selección

In [19]:
file = '../input/draft.txt'
draft_info = pd.read_csv(file)
draft_info.head()

,Rk,Year,Lg,Rd,Pk,Tm,Player,Age,Pos,Born,...,TRB,AST,STL,BLK,FG%,2P%,3P%,FT%,WS,WS/48
0,1,2018,NBA,1,1,PHO,Deandre Ayton\aytonde01,19.338,C,bs,...,10.3,1.8,0.9,0.9,0.585,0.588,0.000,0.746,5.8,0.128
1,2,2018,NBA,1,2,SAC,Marvin Bagley\baglema01,19.104,F,us,...,7.6,1.0,0.5,1.0,0.504,0.534,0.313,0.691,3.6,0.110
2,3,2018,NBA,1,3,ATL,Luka Doncic\doncilu01,19.118,G-F,si,...,7.8,6.0,1.1,0.3,0.427,0.503,0.327,0.713,4.9,0.101
3,4,2018,NBA,1,4,MEM,Jaren Jackson\jacksja02,18.284,F,us,...,4.7,1.1,0.9,1.4,0.506,0.553,0.359,0.766,3.3,0.105
4,5,2018,NBA,1,5,DAL,Trae Young\youngtr01,19.280,G,us,...,3.7,8.1,0.9,0.2,0.418,0.477,0.324,0.829,3.3,0.062


In [20]:
# Esta función separa el nombre del jugador del id que tiene en la web de la NBA
# Entra el DF leído y separa la columna Player en el nombre y el id que da la web de la NBA

def name_split(file):
    Name=[]
    Player_id=[]
    for player in draft_info['Player']:
        Name.append(player.split('\\')[0])
        try:
            Player_id.append(player.split('\\')[1])
        except IndexError:
            Player_id.append("None")
    draft_info['Name'] = Name
    draft_info['Player_id'] = Player_id
    draft_info.drop(['Player'], axis=1, inplace=True)
    return draft_info



In [21]:
draft_info = name_split(draft_info)

In [22]:
draft_info = draft_info.dropna()

In [23]:
draft_info.head()

,Rk,Year,Lg,Rd,Pk,Tm,Age,Pos,Born,College,...,STL,BLK,FG%,2P%,3P%,FT%,WS,WS/48,Name,Player_id
0,1,2018,NBA,1,1,PHO,19.338,C,bs,University of Arizona,...,0.9,0.9,0.585,0.588,0.000,0.746,5.8,0.128,Deandre Ayton,aytonde01
1,2,2018,NBA,1,2,SAC,19.104,F,us,Duke University,...,0.5,1.0,0.504,0.534,0.313,0.691,3.6,0.110,Marvin Bagley,baglema01
3,4,2018,NBA,1,4,MEM,18.284,F,us,Michigan State University,...,0.9,1.4,0.506,0.553,0.359,0.766,3.3,0.105,Jaren Jackson,jacksja02
4,5,2018,NBA,1,5,DAL,19.280,G,us,University of Oklahoma,...,0.9,0.2,0.418,0.477,0.324,0.829,3.3,0.062,Trae Young,youngtr01
5,6,2018,NBA,1,6,ORL,20.045,C,us,University of Texas at Austin,...,0.3,1.4,0.481,0.555,0.300,0.587,1.7,0.106,Mohamed Bamba,bambamo01


In [24]:
draft_info.columns

Index(['Rk', 'Year', 'Lg', 'Rd', 'Pk', 'Tm', 'Age', 'Pos', 'Born', 'College',
       'From', 'To', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%',
       '2P%', '3P%', 'FT%', 'WS', 'WS/48', 'Name', 'Player_id'],
      dtype='object')

In [25]:
remove_columns = ['Rk', 'Lg', 'Rd', 'Tm', 'Age', 'Born', 'College',
       'From', 'To', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%',
       '2P%', '3P%', 'FT%', 'WS', 'WS/48', 'Player_id']

for i in remove_columns:
    draft_info.drop([i], axis=1, inplace=True)

In [26]:
draft_info.columns

Index(['Year', 'Pk', 'Pos', 'Name'], dtype='object')

In [27]:
college_NBAplayers.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SOS,Allstar
0,Greg Monroe,Georgetown,65.0,65.0,32.6,5.3,9.8,0.543,5.2,9.3,...,6.1,8.2,3.2,1.5,1.5,2.9,2.5,14.5,9.26,No
1,Jarron Collins,Stanford,129.0,34.0,24.5,2.9,5.5,0.520,2.8,5.4,...,4.7,5.5,1.1,0.6,0.4,1.3,1.8,8.4,8.87,No
2,Jordan Adams,UCLA,69.0,63.0,30.2,5.4,11.6,0.467,4.0,7.3,...,3.3,4.6,2.1,2.4,0.2,1.4,2.1,16.4,7.72,No
3,Reggie Jackson,Boston College,99.0,52.0,28.0,4.6,9.9,0.465,3.4,6.4,...,3.0,4.4,3.5,0.8,0.5,2.1,1.7,12.7,7.25,No
4,James Webb,Boise State,63.0,58.0,29.0,5.0,9.6,0.518,3.8,6.1,...,6.4,8.6,0.8,1.3,0.6,1.3,2.8,13.5,2.25,No


In [28]:
college_nba = college_NBAplayers.merge(draft_info, left_on='Name', right_on='Name')
college_nba.head()

,Name,School,G,GS,MP,FG,FGA,FG%,2P,2PA,...,STL,BLK,TOV,PF,PTS,SOS,Allstar,Year,Pk,Pos
0,Greg Monroe,Georgetown,65.0,65.0,32.6,5.3,9.8,0.543,5.2,9.3,...,1.5,1.5,2.9,2.5,14.5,9.26,No,2010,7,C-F
1,Jarron Collins,Stanford,129.0,34.0,24.5,2.9,5.5,0.520,2.8,5.4,...,0.6,0.4,1.3,1.8,8.4,8.87,No,2001,53,F-C
2,Jordan Adams,UCLA,69.0,63.0,30.2,5.4,11.6,0.467,4.0,7.3,...,2.4,0.2,1.4,2.1,16.4,7.72,No,2014,22,G
3,Reggie Jackson,Boston College,99.0,52.0,28.0,4.6,9.9,0.465,3.4,6.4,...,0.8,0.5,2.1,1.7,12.7,7.25,No,2011,24,G
4,Tony Snell,New Mexico,104.0,76.0,24.8,3.0,7.1,0.421,1.4,2.9,...,0.5,0.3,1.5,1.2,9.2,4.95,No,2013,20,G-F


In [29]:
college_NBAplayers['Allstar'].value_counts()

No     837
Yes     60
Name: Allstar, dtype: int64

In [30]:
college_nba['Allstar'].value_counts()

No     514
Yes     60
Name: Allstar, dtype: int64

In [31]:
college_nba.drop(['School'], axis=1, inplace=True)

In [32]:
columns = ['Name', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA',
       '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SOS', 'ALLSTAR', 'YEAR', 'PK',
       'POS']

college_nba.columns = columns

In [33]:
college_nba['POS'].value_counts()

G      254
F      179
G-F     42
F-C     34
C       32
F-G     17
C-F     16
Name: POS, dtype: int64

In [34]:
# Creamos una función que nos devuelve la posición principal de un jugador en el caso de tener dos en la base de datos.

def position_split(college_nba):
    position=[]
    for pos in college_nba['POS']:
        position.append(pos.split('-')[0])
    college_nba['POS']=position
    return college_nba

college_nba = position_split(college_nba)

def position_name(college_nba):
    position=[]
    for pos in college_nba['POS']:
        if pos=='G':
            position.append('Guard')
        if pos=='F':
            position.append('Forward')
        if pos=='C':
            position.append('Center')
    college_nba['POS']=position
    return college_nba

college_nba = position_name(college_nba)
college_nba.head()

,Name,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,...,STL,BLK,TOV,PF,PTS,SOS,ALLSTAR,YEAR,PK,POS
0,Greg Monroe,65.0,65.0,32.6,5.3,9.8,0.543,5.2,9.3,0.558,...,1.5,1.5,2.9,2.5,14.5,9.26,No,2010,7,Center
1,Jarron Collins,129.0,34.0,24.5,2.9,5.5,0.520,2.8,5.4,0.527,...,0.6,0.4,1.3,1.8,8.4,8.87,No,2001,53,Forward
2,Jordan Adams,69.0,63.0,30.2,5.4,11.6,0.467,4.0,7.3,0.548,...,2.4,0.2,1.4,2.1,16.4,7.72,No,2014,22,Guard
3,Reggie Jackson,99.0,52.0,28.0,4.6,9.9,0.465,3.4,6.4,0.527,...,0.8,0.5,2.1,1.7,12.7,7.25,No,2011,24,Guard
4,Tony Snell,104.0,76.0,24.8,3.0,7.1,0.421,1.4,2.9,0.480,...,0.5,0.3,1.5,1.2,9.2,4.95,No,2013,20,Guard


In [35]:
#remove_columns = ['G', 'GS']

#for i in remove_columns:
#    college_nba.drop([i], axis=1, inplace=True)

In [36]:
college_nba['POS'].value_counts()

Guard      296
Forward    230
Center      48
Name: POS, dtype: int64

### Exportación del archivo en formato .csv

In [37]:
# Salvamos el archivo

college_nba.to_csv('../input/college_nba.csv', index=False)